In [ ]:
# Import libraries
!pip install yfinance
!pip install fredapi


import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from datetime import datetime, timedelta
from scipy.optimize import minimize
from fredapi import Fred

# Section 1: Define Tickers and Time Range

In [3]:
tickers = ['AMZN', 'VTI', 'VWO', 'VEA', 'BND', 'BJ', 'GS', 'IWM', 'FTNT']

In [ ]:
# Date setting 

end_date = datetime.today() # Set to today

start_date = end_date - timedelta(days = 5*30)  # Set to 5 years ago

print(start_date)

# Section 2: Download Adjusted Close Prices

In [ ]:
adj_close_df = pd.DataFrame()
display(adj_close_df)

In [ ]:
for ticker in tickers:
    data = yf.download(ticker, start = start_date, end = end_date)
    adj_close_df[ticker] = data['Adj Close']

# Section 3: Calculate Lognormal Returns

In [7]:
# Calculate the lognormal returns for each ticker 

log_returns = np.log(adj_close_df / adj_close_df.shift(1))
log_returns = log_returns.dropna()   # Cleanse data with missing values

# Section 4: Calculate CoVariance Matrix

In [ ]:
cov_matrix = log_returns.cov() * 252 # Use 252 because it is trading days during the year
print(cov_matrix)   

# Section 5: Define Portfolio Performance Metrics 

In [9]:
# Calculate the portfolio standard deviation
def std_dev(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)


In [10]:
# Calculate the expected return

def ept_return(weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252

In [11]:
# Calculate the Sharpe Ratio
# Sharpe Ratio = (Portfolio Return - Risk Free Rate) / St.Dev

def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return(ept_return(weights, log_returns) - risk_free_rate) / std_dev(weights, cov_matrix)

# Section 6: Portfolio Optimization

In [12]:
# Set the risk-free rate according to textbook

# risk_free_rate = 0.02

In [ ]:
# Get API from the FED

fred = Fred(api_key = '87f6fd22bd418ae0d91a107ad639d347')
ten_year_treasury_rate = fred.get_series_latest_release('GS10') / 100

# Set the risk-free rate

risk_free_rate = ten_year_treasury_rate.iloc[-1]

# print(ten_year_treasury_rate)
print(risk_free_rate)

In [14]:
# Define the function to minimize

# Negative Sharpe Ratio
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -(ept_return(weights, log_returns) - risk_free_rate) / std_dev(weights, cov_matrix)

In [ ]:
# Set the constraints and bounds
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.3) for _ in range(len(tickers))] # List of tuples that set the lower and upper bounds for each weight.



In [ ]:
# Set initial weights

initial_weights = np.array([1/len(tickers)]*len(tickers))

display(initial_weights)

In [17]:
# Optimize the weights to maximize Sharpe Ratio

optimized_results = minimize(neg_sharpe_ratio, 
                             initial_weights, 
                             args=(log_returns, cov_matrix, risk_free_rate), 
                             method='SLSQP', 
                             constraints=constraints, 
                             bounds=bounds)

In [18]:
# Get the optimal weights

optimal_weights = optimized_results.x


# Section 7: Analyze the Optimal Portfolio

In [ ]:
print("Optimal Weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

optimal_portfolio_return = ept_return(optimal_weights, log_returns)
optimal_portfolio_volatility = std_dev(optimal_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")

# Section 8: Data visualization

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.bar(tickers, optimal_weights)

plt.xlabel('Assets')
plt.ylabel('Optimal Weights')
plt.title('Optimal Portfolio Weights')

plt.show()